<a href="https://colab.research.google.com/github/yf591/sd-model-merge-tool/blob/main/04_Merge_Model_Maker_Ver2_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3モデル以上単純マージ（Huggin Face, Civitai, MyDriveからのロードに対応）

## 事前準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ### ライブラリのインストールと準備

from google.colab import output

# Hugging Face Hub, PyTorch, その他必要なライブラリをインストール
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # PyTorchを使用して深層学習モデルを操作します。CUDAバージョン（例: `cu118`）を指定
!pip install diffusers transformers accelerate # Stable Diffusionを扱うための主要ライブラリです。モデルのロードや画像生成の操作を簡素化
!pip install safetensors # 安全かつ軽量なモデル保存形式（`.safetensors`）をサポート
!pip install huggingface-hub # Hugging Face Hubからモデルをダウンロード・管理
!pip install opencv-python # 生成した画像の前処理や後処理に使用
!pip install numpy # 数値計算ライブラリで、モデルや画像の操作に使う
!pip install matplotlib # 生成された画像の可視化に使う
!pip install tqdm # プログレスバーの表示
!pip install optuna # ハイパーパラメータ最適化
!pip install requests

output.clear()

In [ ]:
#@title ### 必要なライブラリのインポート

import os
import torch
from safetensors.torch import load_file, save_file
from transformers import AutoConfig, AutoModel
from diffusers import DiffusionPipeline, StableDiffusionPipeline, UNet2DConditionModel
import shutil
from huggingface_hub import hf_hub_download
from typing import List, Dict
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import PIL.Image
import numpy as np
import requests
from tqdm import tqdm
import uuid

In [ ]:
#@title ### APIキー設定（Hugging Face, Civitai）

from getpass import getpass
from google.colab import userdata

# Hugging Faceで取得したTokenをこちらに貼る(トークンを非表示で入力)
HF_TOKEN = getpass("Hugging FaceのRead権限のあるHF Tokenを入力してください: ")

# CIVITAI_TOKEN が存在する場合、取得
api_key = userdata.get('CIVITAI_TOKEN')
if api_key is None:
    print("Error: CIVITAI_API_KEY secret is not set.")

## 各種設定（各種関数の定義、モデル数の設定）

In [ ]:
#@title ### 関数の定義（モデルのダウンロードとロード）

def download_model(repo_id, filename, token):
    """Hugging Face Hubからモデルをダウンロード"""
    return hf_hub_download(repo_id=repo_id, filename=filename, token=token)

def download_civitai_model(url, output_path, api_key):
    """Civitaiからモデルをダウンロード"""
    try:
        headers = {"Authorization": f"Bearer {api_key}"} if api_key else None
        response = requests.get(url, stream=True, headers=headers)
        response.raise_for_status()

        total_size = int(response.headers.get('content-length', 0))
        with open(output_path, 'wb') as file, tqdm(
            desc=output_path,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
             for data in response.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)
        return output_path
    except Exception as e:
        print(f"Error downloading from Civitai: {e}")
        return None


def load_model(path, device, api_key=None):
    """ファイルパスまたはURLからモデルをロードする"""
    try:
        if path.startswith("http"):
            # URLの場合（Civitaiなど）は、ダウンロードしてから読み込む
            if "civitai.com" in path:
                unique_id = str(uuid.uuid4())
                output_path = f"/content/downloaded_models/model_{unique_id}.safetensors"
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                downloaded_path = download_civitai_model(path, output_path, api_key)
                if not downloaded_path:
                   return None
                else:
                   print(f"Civitaiからモデルをロード: {downloaded_path}")
                   try:
                       return load_file(downloaded_path, device=device)
                   except Exception as e:
                     print(f"Error loading downloaded Civitai model: {e}. Attempting to redownload...")
                     os.remove(downloaded_path)
                     downloaded_path = download_civitai_model(path, output_path, api_key)
                     if not downloaded_path:
                        return None
                     return load_file(downloaded_path, device=device)
            elif "huggingface.co" in path:
                print(f"HuggingFaceからモデルをロード: {path}")
                repo_id_and_file = path.split("huggingface.co/")[1]
                repo_id = repo_id_and_file.split("/resolve/")[0]
                filename = repo_id_and_file.split("/")[-1]
                path = download_model(repo_id, filename, HF_TOKEN)
                return load_file(path, device=device)
            else:
                print("Error: HTTP URL not recognized, use HuggingFace or Civitai Model.")
                return None

        if path.startswith("/content/drive"):
          # Google Drive のパスの場合
            print(f"Google Driveからモデルをロード: {path}")
            return load_file(path, device=device)
        else:
          print("Error: Incorrect Model Path.")
          return None
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [ ]:
#@title ### 関数の定義（モデルのマージとテスト、保存関数）

def merge_multiple_models(models: List[Dict], alpha):
    """複数のモデルを単純マージ（テンソルサイズが異なる場合を処理）"""
    merged_weights = {}

    # 初期モデルのキー構造を取得
    base_model_keys = set(models[0]['weights'].keys())

    for key in base_model_keys:
        weights_to_merge = []
        valid_alphas = []

        for model in models:
            weights = model['weights']
            if key in weights:
                # テンソルサイズの一致を確認
                if weights[key].size() == models[0]['weights'][key].size():
                    weights_to_merge.append(weights[key])
                    valid_alphas.append(model['alpha'])
                else:
                    print(f"警告: レイヤー {key} のサイズが一致しません。スキップします。")

        if weights_to_merge:
            merged_weights[key] = sum(
                alpha * weight for alpha, weight in zip(valid_alphas, weights_to_merge)
            )
        else:
            # レイヤー構造が一致しない場合、最初のモデルの重みを使用
            print(f"情報: レイヤー {key} に対応する重みが見つからないため、最初のモデルの重みを使用します。")
            merged_weights[key] = models[0]['weights'][key]

    return merged_weights

def test_model_memory(merged_weights, test_function):
 """メモリ上のモデルでテスト"""
 try:
     test_results = test_function(merged_weights)
     print("テスト結果:", test_results)
     if test_results and test_results.startswith("テスト成功:"):
         display(Image(filename=test_results.split(":",1)[1].strip()))
 except Exception as e:
     print(f"テスト中にエラーが発生しました: {e}")

def save_merged_model(merged_weights, output_path):
    """マージ済みモデルを保存"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_file(merged_weights, output_path)

In [ ]:
#@title ### Diffusers形式での保存関数

def save_merged_weights_as_diffusers_format(merged_weights, output_dir):
    """マージ済みの重みをDiffusers形式で保存"""
    os.makedirs(output_dir, exist_ok=True)
    # UNetモデルのコンフィグを取得
    config = AutoConfig.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", force_download=True)
    # configにmodel_typeを追加
    config.model_type = "unet"
    # コンフィグからUNet2DConditionModelを生成
    model = UNet2DConditionModel.from_config(config)
    # マージされた重みを適用
    model.load_state_dict(merged_weights, strict=False)
    # Diffusers形式でモデルを保存
    model.save_pretrained(output_dir)
    print(f"マージ済みモデルを {output_dir} に保存しました。")

In [ ]:
#@title ### テスト用画像生成関数

def example_test_function(weights):
    """画像生成をテストする関数"""
    try:
        positive_prompt = positive_prompt_widget.value
        negative_prompt = negative_prompt_widget.value

        print("ポジティブプロンプト:", positive_prompt)
        print("ネガティブプロンプト:", negative_prompt)

        with torch.no_grad():
            # UNetモデルのコンフィグを取得
            config = AutoConfig.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", force_download=True)
            # コンフィグからUNet2DConditionModelを生成
            unet = UNet2DConditionModel.from_config(config)
            unet.load_state_dict(weights, strict=False)

            # パイプラインをロード
            pipe = DiffusionPipeline.from_pretrained(
                "CompVis/stable-diffusion-v1-4",
                unet=unet,
                torch_dtype=torch.float32,
                safety_checker=None
            ).to("cuda" if torch.cuda.is_available() else "cpu")

            pipe.enable_xformers_memory_efficient_attention()

            # 画像生成
            image = pipe(
                prompt=positive_prompt,
                negative_prompt=negative_prompt,
                guidance_scale=7.5,
                num_inference_steps=10,
                width=128,
                height=128,
            ).images[0]
            del unet
            torch.cuda.empty_cache()
            gc.collect()


        # 生成画像を表示
        temp_path = "/content/temp_test_image.png"
        image.save(temp_path)

        return f"テスト成功: {temp_path}"
    except Exception as e:
        return f"テスト失敗: {e}"

In [ ]:
#@title MergeするModel数の定義

# UI設定
num_models = 3 #@param {type:"integer"}

# パスとアルファ値を格納するリストを初期化
paths = []
sliders = []
alpha_n = None

# パス入力UIの生成
for i in range(num_models):
    paths.append(widgets.Text(value="", description=f"Path{i+1}", layout=widgets.Layout(width='80%')))

# アルファ値設定UIの生成
if num_models > 1:
    sliders = [widgets.FloatSlider(value=1/num_models, min=0, max=1, step=0.01, description=f"Alpha{i+1}") for i in range(num_models-1)]
    alpha_n = widgets.FloatText(value=1-sum([slider.value for slider in sliders]), description=f"Alpha{num_models}", disabled = True)
else:
    alpha_n = widgets.FloatText(value=1, description="Alpha1", disabled = True)

def enforce_alpha_constraints(*args):
    total_alpha = sum(slider.value for slider in sliders)
    if total_alpha > 1.0:
         for slider in sliders:
            slider.value = slider.value / total_alpha
    alpha_n.value = 1 - total_alpha

for slider in sliders:
    slider.observe(enforce_alpha_constraints, 'value')

output_file_widget = widgets.Text(value="/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/merged_model.safetensors", description="Output", layout=widgets.Layout(width='80%'))

positive_prompt_widget = widgets.Text(value="extremely detailed CG, 8k, masterpiece, best quality, hyperrealistic, sharp focus, intricate details, professional art, perfect lighting, ultra high res, a cute girl in the office, RAW photo, no artifacts, best quality", description="Positive Prompt", layout=widgets.Layout(width='90%'))
negative_prompt_widget = widgets.Text(value="low quality, blurry, pixelated, distorted, bad anatomy, disfigured, out of focus, bad proportions, skin blemishes, low contrast, text, logo, watermark, ((monochrome:1.5)), ((grayscale:1.5)), ((cartoon:1.2)), ((anime:1.2)), ((3d:1.2)), ((skin spots:1.3)), ((acnes:1.3)), ((age spots:1.3))", description="Negative Prompt", layout=widgets.Layout(width='90%'))

## UIの設定と実行

In [ ]:
#@title ### マージ実行関数

def execute_merge(b):
    output_file = output_file_widget.value
    try:
        print("モデルのロードを開始します...")
        models = []
        # 最後のモデルのパスとアルファ値も含める
        all_paths = paths
        all_alphas = sliders + [alpha_n]

        for path, alpha in zip(all_paths, all_alphas):
            if path.value and alpha.value > 0:
                model = load_model(path.value, device="cuda" if torch.cuda.is_available() else "cpu", api_key=api_key)
                if model is not None:
                    models.append({
                        "weights": model,
                        "alpha": alpha.value  # .valueを追加
                    })
                else:
                    print(f"Error: model loading failed. Skip this model.")

        if len(models) == 0:
            print("Error: At least one model is required for merging.")
        else:
            for idx, model in enumerate(models):
                print(f"model{idx + 1} keys: {list(model['weights'].keys())[:5]} ...")

            print("モデルをマージ中...")
            merged_weights = merge_multiple_models(models, alpha=None)

            print("メモリ上のモデルでテスト中...")
            test_model_memory(merged_weights, example_test_function)

            confirm_button = widgets.Button(description="画像が気に入ったら保存")
            retry_button = widgets.Button(description="重みを再調整")

            def on_confirm_clicked(b):
                print(f"マージ済みモデルを保存します: {output_file}")
                save_merged_model(merged_weights, output_file)
                print("マージ完了！")
                confirm_button.close()
                retry_button.close()
                return

            def on_retry_clicked(b):
                print("重みを再調整してください。")
                clear_output()
                display(ui, output_file_widget, merge_button)

            confirm_button.on_click(on_confirm_clicked)
            retry_button.on_click(on_retry_clicked)

            display(confirm_button, retry_button)
            return

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        raise e

In [ ]:
#@title ### UIの表示と実行ボタン

# UIの表示部分を修正
ui = widgets.VBox(paths + sliders + [alpha_n, positive_prompt_widget, negative_prompt_widget])
merge_button = widgets.Button(description="マージ実行")
merge_button.on_click(execute_merge)
display(ui, output_file_widget, merge_button)